In [52]:
#Adding defensive ratings for each player by year...

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
from bs4 import BeautifulSoup, Comment


In [5]:
box = pd.read_csv("../data/BoxScoreData.csv")
player_ids = box.Player_ID.unique()

In [54]:
head_options = webdriver.ChromeOptions()
head_options.add_argument("--headless")
head_options.add_argument("--disable-blink-features=AutomationControlled")
head_options.add_argument("--disable-gpu")
head_options.add_argument("--no-sandbox")
head_options.add_argument("--disable-extensions")
head_options.add_argument("--disable-dev-shm-usage")
head_options.add_argument("--disable-popup-blocking")
head_options.add_argument("--disable-notifications")
head_options.add_argument("--disable-infobars")
head_options.add_experimental_option("excludeSwitches", ["enable-automation"])
head_options.add_experimental_option('useAutomationExtension', False)
prefs = {"profile.managed_default_content_settings.images": 2}
head_options.add_experimental_option("prefs", prefs)


driver = webdriver.Chrome(options=head_options)

df = pd.DataFrame(data = [], columns = ["player_id", "Season", "Age", "Team", "Pos", "ortg", "drtg", "awards"])

for id in player_ids:
    time.sleep(random.uniform(0.1,1))
    url = f"https://www.basketball-reference.com/players/{id[0]}/{id}.html"
    driver.get(url)



        
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # First, check if there's a normal visible table
    table = soup.find("table", id="per_poss")

    # If not visible, look inside HTML comments
    if not table:
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        for c in comments:
            if 'per_poss' in c:
                comment_soup = BeautifulSoup(c, "html.parser")
                table = comment_soup.find("table", id="per_poss")
                if table:
                    break

    if not table:
        print(f"⚠️ No per_poss table found for {id}")
        continue

    # Parse rows
    rows = table.find_all("tr")
    seen = set()

    for row in rows:
        th = row.find("th", {"data-stat": "year_id"})
        if not th or not th.text.strip():
            continue  # skip header or empty rows

        season = th.text.strip().split('-')[0]  # "1999-00" → "1999"
        cols = row.find_all("td")
        if len(cols) < 31:
            continue

        age = cols[0].text.strip()
        team = cols[1].text.strip()
        pos = cols[2].text.strip()
        ortg = cols[29].text.strip()
        drtg = cols[30].text.strip()
        awards = cols[31].text.strip() if len(cols) > 31 else ""

        print(age)
        key = (season, team)
        if key in seen:
            continue
        seen.add(key)

        df.loc[len(df)] = [id, season, age, team, pos, ortg, drtg, awards]


    # #print(url)
    # rows = []
    # try:
    #     adv_box = driver.find_element(By.ID, "per_poss")

    #     rows = adv_box.find_elements(By.TAG_NAME, "tr")
    # except Exception as e:
    #     print("fucked")

    # prev_age = -1 #This is used to not have excess rows when a player switches team mid-season

    # for row in rows:
    #     cols = row.find_elements(By.TAG_NAME, "td")
    #     if len(cols) > 31:
    #         #What actually is necessary here?
    #         if prev_age == cols[0].text:
    #             continue #We doubled up
    #         season_cell = row.find_element(By.CSS_SELECTOR, 'th[data-stat="year_id"]')
    #         season = season_cell.text.strip()         
    #         season = f"20{season[-2:]}"
    #         prev_age = cols[0].text
    #         team = cols[1].text
    #         pos = cols[2].text
    #         ortg = cols[29].text
    #         drtg = cols[30].text
    #         awards = cols[31].text

    #         df.loc[len(df)] = [id, season, prev_age, team, pos, ortg, drtg, awards]


print(df)

⚠️ No per_poss table found for chandty01
⚠️ No per_poss table found for douglto01
⚠️ No per_poss table found for stoudam01
⚠️ No per_poss table found for fieldla01
⚠️ No per_poss table found for anthoca01
⚠️ No per_poss table found for shumpim01
⚠️ No per_poss table found for walkebi01


KeyboardInterrupt: 

In [50]:

driver = webdriver.Chrome()

driver.get(url)
